# Data Exploration Projet: Vorhersage von Diabetes

Bevor das Projekt ausgeführt werden muss, muss eine virtuelle Umgebung erstellt und aktiviert werden.


1. Um eine virtuelle Umgebung zu erstellen, kann folgender Befehl in Terminal ausgeführt werden:

In [ ]:
python3 -m venv .venv_dep

2. Daher wird ein neuer Ordner in dem Verzeichnis automatisch erstellt.

3. Als Nächstes muss die Umgebung mit folgendem Befehl aktiviert werden:

In [ ]:
source .venv_dep/bin/activate

oder in VSCode muss man manuell die Kernel auf oben rechte Seite festlegen.

4. Jetzt ist alles vorbereitet.

In [ ]:
# Install Dependencies

%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install matplotlib
%pip install mlflow
%pip install seaborn
%pip install mlxtend
%pip install graphviz

In [ ]:
#Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions


%matplotlib inline

# **Auswahl Datensatz**

In [ ]:
#Import Dataset
dataset = "https://raw.githubusercontent.com/rakaputra12/Data_Exploration_Diabetes/main/Healthcare-Diabetes.csv"
diabetes_df = pd.read_csv(dataset)

Der Datensatz ist in dem GitHub-Repository abgelegt, was bedeutet, dass er von überall aus einfach zugänglich ist. Dazu wird auch der Datensatz in dem Ordner dazu gegeben, aber muss die Pfad angepasst werden.

In [ ]:
diabetes_df.head()

In [ ]:
#Delete the column "Id"
diabetes_df = diabetes_df.drop('Id', axis=1)

# **Charakterisierung des Datensatzes**

In [ ]:
diabetes_df.columns

In [ ]:
diabetes_df.info()

In [ ]:
diabetes_df.describe()

In [ ]:
diabetes_df.describe().T

In [ ]:
diabetes_df.isnull()

In [ ]:
diabetes_df.isnull().sum()

Im Rohdatensatz, bevor eine Bereinigung durchgeführt wird, ist es nicht ungewöhnlich, fehlende Werte zu finden.Daher ist es in dem Fall wichtig, eine gründliche Untersuchung der fehlenden Werte durchzuführen, um ihr Ausmaß und ihre Verteilung zu verstehen. 

In [ ]:
zero_counts = (diabetes_df == 0).sum()
zero_counts

After further checking, it was found that in certain columns, the zero values did not make sense, indicating missing values.

Following columns or variables have an invalid zero value: Glucose, BloodPressure, SkinThickness, Insulin, BMI. There are many approachs to handling missing values in a dataset. Replacing zero values with NaN and then filling them with the distribution for each column is one of many common possibilities.

Replacing zero values with NaN and then filling them with the distribution for each column is a reasonable approach to handling missing data in this case, cause Reducing the potential bias introduced by using a single imputation method for all missing values leads to more accurate and robust analyses, particularly in datasets where different columns have distinct distributions.This technique allows you to preserve the integrity of the data and capture the uncertainty associated with missing values. On the column like Pregnancies, a value of zero does make sense, cause someone can have never been pregnant. For the column "Outcome" is because the value for sufferer or not is only differentiated by one or zero.

Bei einer weiteren Überprüfung wurde festgestellt, dass in bestimmten Spalten die Nullwerte keinen Sinn ergaben, was auf fehlende Werte hindeutet.
Die folgenden Spalten oder Variablen haben einen ungültigen Nullwert: Glukose, Blutdruck, Hautdicke, Insulin, BMI. Es gibt viele Ansätze für die Behandlung fehlender Werte in einem Datensatz. Das Ersetzen von Nullwerten durch NaN und das anschließende Auffüllen mit der Verteilung für jede Spalte ist eine von vielen gängigen Möglichkeiten.
Das Ersetzen von Nullwerten durch NaN und das anschließende Auffüllen mit der Verteilung für jede Spalte ist in diesem Fall ein vernünftiger Ansatz für den Umgang mit fehlenden Daten, denn die Verringerung der potenziellen Verzerrung durch die Verwendung einer einzigen Imputationsmethode für alle fehlenden Werte führt zu genaueren und robusteren Analysen, insbesondere in Datensätzen, in denen verschiedene Spalten unterschiedliche Verteilungen aufweisen. In der Spalte "Schwangerschaften" macht ein Wert von Null durchaus Sinn, da jemand nie schwanger gewesen sein kann. Bei der Spalte "Ergebnis" wird der Wert für "erkrankt" oder "nicht erkrankt" nur durch eins oder null unterschieden.

In [ ]:
#Replace zeros with NaN Value
filtered_diabetes_df = diabetes_df.copy(deep = True)
filtered_diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = filtered_diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0,np.NaN)

#Showing the Count of NaNs
print(filtered_diabetes_df.isnull().sum())

In [ ]:
#Aiming to impute NaN values for the columns in accordance with their distribution
fill_values = {
    'Glucose': filtered_diabetes_df['Glucose'].mean(),
    'BloodPressure': filtered_diabetes_df['BloodPressure'].mean(),
    'SkinThickness': filtered_diabetes_df['SkinThickness'].mean(),
    'Insulin': filtered_diabetes_df['Insulin'].mean(),
    'BMI': filtered_diabetes_df['BMI'].mean()
}
filtered_diabetes_df.fillna(value=fill_values, inplace=True)

In [ ]:
#Showing the Count of NaNs
print(filtered_diabetes_df.isnull().sum())

Nun ist der Datensatz bereit zu verarbeiten.

## **Perform EDA (Exploratory Data Analysis )**

In [ ]:
#Checking the balance of the data by plotting the count of outcomes by their values
colors = ['#0392cf', '#7bc043']

print(filtered_diabetes_df.Outcome.value_counts())
p = filtered_diabetes_df.Outcome.value_counts().plot(kind="bar", color = colors)

In [ ]:
# Plotting a pie chart to visualize the percentage distribution of diabetes and non-diabetes outcomes
filtered_diabetes_df['Outcome'].value_counts().plot(kind='pie',autopct='%1.1f%%')
plt.title('Prozentualer Anteil von Diabetes gegenüber Nicht-Diabetes')
plt.show()

The above graph shows that the data is biased towards datapoints having outcome value as 0 where it means that diabetes was not present actually. The number of non-diabetics is almost twice the number of diabetic patients.

Das obige Diagramm zeigt, dass die Daten leicht in Richtung der Datenpunkte mit einem Rückgabewert von 0 verzerrt sind. Die Zahl der Nicht-Diabetiker ist fast doppelt so hoch wie die Zahl der Diabetiker.

In [ ]:
#Plotting the Pair Plots for the cleaned data
p = sns.pairplot(filtered_diabetes_df, hue= 'Outcome')

In [ ]:
#Plotting the distribution after replacing the NaN Values
p = filtered_diabetes_df.hist(figsize=(15,15))

## **Correlation between all the features**

In [ ]:
# Calculate the correlation matrix 
filtered_diabetes_df.corr()

In [ ]:
#Correlation between all the features after cleaning
plt.figure(figsize= (12,10))
p = sns.heatmap(filtered_diabetes_df.corr(), annot = True, cmap = 'RdYlGn')

# **Feature Engineering**

In [ ]:
#Import Libraries for Feature Enginnering
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Extracting features (X) and target variable (y) from the DataFrames
X=filtered_diabetes_df[filtered_diabetes_df.columns[0:-1]]
y=filtered_diabetes_df[filtered_diabetes_df.columns[-1]]

### **Split the dataset for Feature Importance**

In [ ]:
# Splitting the dataset into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,random_state=42)

## **Ermittlung der Feature Importance mit einem Decision Tree**

In [ ]:
# Instantiate a DecisionTreeClassifier with specified parameters
tree = DecisionTreeClassifier(max_depth=4,random_state=0)

# Train the Decision Tree classifier on the training data
tree.fit(X_train,y_train)

# Print the accuracy on the test set and the training set
print("Accuracy on training set: {:.3f}".format(tree.score(X_train,y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test,y_test)))

In [ ]:
from sklearn.tree import export_graphviz

# Export the decision tree visualization to a .dot file
export_graphviz(tree,
                out_file="diabetes_tree.dot",
                class_names=["0","1"],
                feature_names=X.columns,
                impurity=False,
                filled=True)

import graphviz

# Read the .dot file and visualize the decision tree
with open("diabetes_tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

In [ ]:
print("Feature importances:\n{}".format(tree.feature_importances_))

In [ ]:
# Create a DataFrame to display feature importances
df_feature_importance = pd.DataFrame({'Feature Names': X.columns, 'Importance of  Feature': tree.feature_importances_ })
df_feature_importance

In [ ]:
# Plot feature importances using the provided decision tree model
def plot_feature_importances_with_DecisionTree(model):
    n_features = X.shape[1]
    plt.figure(figsize=(10, 6))
    plt.barh(range(n_features),model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.savefig("feature_importance_with_DecisionTree.png", dpi=300)
    plt.show()
    plt.close()
plot_feature_importances_with_DecisionTree(tree)

Hier kann man erkennen, dass Feature "Glucose", "BMI" and "Age" Parameter sind, auf die geachtet werden müssen.

## **Ermittlung der Feature Importance mit einem Random  (Als Vergleich)**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define feature names
feature_names = [f"feature {i}" for i in range(X.shape[1])]

# Instantiate a RandomForestClassifier with specified parameters
forest = RandomForestClassifier(n_estimators=100,random_state=0)

# Train the Random Forest classifier on the training data
forest.fit(X_train, y_train)

In [ ]:
print(forest.feature_importances_)

In [ ]:
# Create a DataFrame to display feature importances
df_feature_importance_withRandomForest = pd.DataFrame({'Feature Names': X.columns, 'Importance of  Feature': forest.feature_importances_ })
df_feature_importance_withRandomForest

In [ ]:
def plot_feature_importances_with_RandomForest(model):
    n_features = X.shape[1]
    plt.figure(figsize=(10, 6))
    plt.barh(range(n_features),model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.savefig("feature_imporatnace_with_RandomForest.png", dpi=300)
    plt.show()
    plt.close()
plot_feature_importances_with_RandomForest(forest)

Nach einem Vergleich mit dem RandomForest-Algorithmus wurde bestätigt, dass die Merkmale "Glucose", "BMI" und "Age" als die wichtigsten Merkmale identifiziert wurden. Dies deutet darauf hin, dass diese spezifischen Merkmale einen signifikanten Einfluss auf das Modell haben und einen wesentlichen Beitrag zur Vorhersageleistung leisten. 

# **Auswahl der Metriken**

Für den Datensatz "Diabetes" wurde dazu entschieden, den F1-Score als Evaluationsmetrik zu verwenden. Der F1-Score ist eine Metrik, die das Gleichgewicht zwischen Präzision und Rückruf (Recall) misst. Es ist besonders nützlich, wenn die Klassen im Datensatz nicht gleichmäßig verteilt sind oder wenn False Positives und False Negatives ähnlich schwerwiegend sind. Im Falle des Diabetes-Datensatzes, bei dem es um die korrekte Vorhersage von Diabetesfällen geht, kann der F1-Score hilfreich sein, um sicherzustellen, dass sowohl die Präzision als auch der Rückruf bei der Vorhersage von positiven Fällen berücksichtigt werden.

Es gibt verschiedene Überlegungen und Prioritäten für jede Metrik bei F1-Score:

- Micro F1-Score:

    - Der Micro F1-Score aggregiert die True Positives, False Positives und False Negatives über alle Klassen und berechnet dann den F1-Score.
    - Da er die globale Anzahl der Vorhersagen für jede Klasse berücksichtigt, ist er geeignet, wenn die Klassen unterschiedlich groß sind.
    - Dies könnte nützlich sein, wenn es sichergestellt wird, dass das Modell gut auf die insgesamt am häufigsten auftretende Klasse (normalerweise die negative Klasse) reagiert.

- Weighted F1-Score:

    - Der Weighted F1-Score berücksichtigt die Ungleichheit der Klassen, indem er den F1-Score für jede Klasse berechnet und dann einen gewichteten Durchschnitt basierend auf der Klassegröße nimmt.
    - Da er die Klassen entsprechend ihrer Häufigkeit im Datensatz gewichtet, ist er gut geeignet, wenn die Klassen im Datensatz ungleich verteilt sind.
    - Dies könnte relevant sein, wenn gleiche Leistung für beide Klassen gewünscht werden oder wenn die positive Klasse (Diabetes positiv) wichtiger ist und es sichergestellt werden möchte, dass das Modell gute Ergebnisse für diese Klasse liefert.


In dem Fall wird die Metrik "Weighted F1-Score" angewendet, weil hinsichtlich der Charakterisierung des Datensatz die Klasse ungleicht verteilt sind und die Klasse größenteils für negative Klassen sind, daher müssen positive Klassen vorangetrieben und sind wichtiger. Dazu wird Macro F1-Score nicht berücksichtigt, weil er eine gutel Wahl, wenn alle Klasse gleich sind.



# **Auswahl und Beschreibung der ML-Methode**

Dazu kommt die Methode "Supervised Learning" zum Einsatz. Dafür wurde auch der Support Vector Machine (SVM) Algorithmus in Betracht gezogen. 

Die Support Vector Machine ist ein leistungsstarker Algorithmus für die Klassifizierung und Regression, der besonders effektiv in der Verarbeitung komplexer Daten und in Szenarien mit hoher Dimensionalität ist. Dies macht sie zu einer attraktiven Wahl für den Diabetes-Datensatz, der oft Merkmale mit hoher Dimensionalität aufweist und möglicherweise nicht-linear trennbar ist.

Ein weiterer Vorteil der SVM ist ihre Fähigkeit, gut mit kleineren Trainingsdatensätzen umzugehen, ohne an Leistung zu verlieren. Dies ist besonders relevant in Situationen, in denen der Diabetes-Datensatz begrenzte Datenpunkte aufweisen könnte.

Darüber hinaus bietet die SVM die Möglichkeit, verschiedene Kernel-Funktionen zu verwenden, um die Entscheidungsgrenze zwischen den Klassen anzupassen. Dies ermöglicht es, auch in komplexen, nicht-linearen Datenstrukturen effektive Trennungen zu finden.

## **Preparation Data**

Bevor das Modell mit Diabetes Datensatz trainiert wird, muss der Datensatz noch ein bisschen vorbereitet werden. Datenpräparation hilft dabei, sicherzustellen, dass die Eingabedaten in der Produktionsumgebung, in der das Modell eingesetzt wird, in der richtigen Form vorliegen und dass das Modell effizient und effektiv arbeiten kann.

Es ist auch wichtig zu merken, dass in dem Fall  alle Features für die Training genommen werden, weil zwei Features sehr zu wenig für die Modelling sind.

In [ ]:
# Separating features (X) and target variable (y) from the DataFrame
X = filtered_diabetes_df.drop("Outcome", axis=1)
y = filtered_diabetes_df['Outcome']

### **Hier geht es um die Verteilung von Daten**

In [ ]:
# Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Normalize the features using StandardScaler
x_normalized = StandardScaler().fit_transform(X)
# Convert the normalized features into a DataFrame
df_x_normalized = pd.DataFrame(x_normalized, columns=X.columns)

# Perform PCA with 2 components
x_pca  = PCA(n_components=2).fit_transform(df_x_normalized.values)
# Convert the PCA results into a DataFrame
df_x_pca = pd.DataFrame(x_pca, columns=range(2))

# Combine the PCA results with the target variable 'y'
df_pca = pd.concat([df_x_pca, y], axis=1)

In [ ]:
ax_jointplot = sns.jointplot(data=df_pca, x=0, y=1, hue="Outcome", kind="hist")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Transform the features using StandardScaler and create a DataFrame with scaled features
X = pd.DataFrame(scaler.fit_transform(X), columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])
X.head()

## **Split Dataset**

In [ ]:
#Split the dataset with size 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Split the dataset with size 0.3
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# **Implementierung Training**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from sklearn import svm

# Instantiate SVM classifier
machine = svm.SVC()

# Train the SVM classifier on the training data
machine.fit(X_train,y_train)

# Predict using the trained SVM classifier
y_pred = machine.predict(X_test)

# Calculate accuracy
clf_acc = accuracy_score(y_pred, y_test)

# Print classification report
print(classification_report(y_test, y_pred))

# Print accuracy of SVM classifier
print("Accuracy SVM: {:.2f}%".format(clf_acc * 100))

# **Hyperparametertuning Verwendung ML Lifecycle Mgt im Code**

Hyperparameter-Tuning ist ein entscheidender Schritt im maschinellen Lernprozess, der darauf abzielt, die Leistung eines Modells weiter zu verbessern, indem die optimalen Werte für die Hyperparameter gefunden werden. Selbst wenn ein Modell bereits eine akzeptable Leistung erzielt hat, wie in dem Fall mit einer Punktzahl von 83,75% von der ersten Implementierung, kann das Tuning der Hyperparameter dazu beitragen, die Leistung weiter zu steigern. 

In dem Fall kommt auch dazu ein ML Lifecylce Management "MlFlow" zum Einsatz. Ein ML Lifecycle Management wird verwendet, um Parameter, Metriken und Ausgabedateien zu protokollieren, wenn der maschinellen Lerncode ausgeführt wird, und um die Ergebnisse später zu visualisieren. 

In [ ]:
#Install Dependencies

%pip install mlflow

In [ ]:
import mlflow
from sklearn.model_selection import GridSearchCV

In [ ]:
# Start an MLflow run
mlflow.start_run()

In [ ]:

# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1.0, 10.0, 100.0],  # Regularization parameter
    'kernel': ['linear', 'rbf','poly', 'sigmoid'],  # Kernel type
    'degree': [2, 3, 4],  # Degree of the polynomial kernel
    'gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    'coef0': [0.0, 0.1, 0.5],  # Independent term in kernel function
    'shrinking': [True, False],  # Whether to use the shrinking heuristic
    'probability': [True, False],  # Whether to enable probability estimates
    'tol': [0.0001, 0.001, 0.01],  # Tolerance for stopping criterion
}


# Create GridSearchCV
grid_search = GridSearchCV(estimator=machine, param_grid=param_grid, n_jobs=6, cv=5, scoring='f1_weighted', refit='f1_weighted')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)


# Log parameters and metrics to MLflow
mlflow.log_metric("best_score", grid_search.best_score_)
# Log the best parameters as a parameter
for key, value in grid_search.best_params_.items():
    mlflow.log_param(f"best_{key}", value)

# Log the trained model
mlflow.sklearn.log_model(grid_search.best_estimator_, "svm_model")


In [ ]:
 # End the  MLflow run
mlflow.end_run()

In [ ]:
# Launch the MLflow UI
!mlflow ui

Um den nächsten Code auszuführen, muss MLFlow ui zuerst durch manuelles Drücken der Stopp-Taste gestoppt werden.

# **Evaluation und Ergebnisdarstellung** | **Predict and Evaluate**

In [ ]:
# Use the best estimator to predict
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test)

# Evaluate the model
clf_acc = accuracy_score(y_pred, y_test)
print(classification_report(y_test, y_pred))
print("Accuracy SVM after GridSearchCV: {:.2f}%".format(clf_acc*100))

WENN SPLITTING 0.2

Nachdem das Hyperparameter-Tuning durchgeführt wurde, konnte die Punktzahl des Modells von 83,75% auf 94,04% gesteigert werden. 

WENN SPLITTING 0.3

Nachdem das Hyperparameter-Tuning durchgeführt wurde, konnte die Punktzahl des Modells von 84,00% auf 93,62% gesteigert werden.




Diese signifikante Verbesserung der Punktzahl zeigt, wie wichtig es ist, die Hyperparameter eines Modells sorgfältig anzupassen. Durch das Feintuning der Hyperparameter konnte das Modell besser auf die spezifischen Anforderungen des Datensatzes abgestimmt werden, was zu einer verbesserten Leistung bei der Klassifizierung von Diabetesfällen führt.


In [ ]:
from sklearn.metrics import confusion_matrix

# Create a confusin matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')

# **Vorhersage-Demo | Testing**

Die Vorhersage-Demo mit dem Modell, das optimale Hyperparameter verwendet, bietet einen praxisnahen Einblick in die Leistungsfähigkeit des Modells. Benutzer können in der Demo neue Eingabedaten eingeben, und das Modell wird basierend auf diesen Daten Vorhersagen treffen. 

In [ ]:
# Create a dictionary containing the new data
new_data = {
      'Pregnancies': [1],
      'Glucose': [120],
      'BloodPressure': [66],
      'SkinThickness': [29],
      'Insulin': [49],
      'BMI': [47.6],
      'DiabetesPedigreeFunction': [0.351],
      'Age': [27]

}

# Create a DataFrame from the dictionary
new_data = pd.DataFrame(new_data)
new_data

In [ ]:
# Scale the new data using the previously fitted StandardScaler
scaled_new_data = pd.DataFrame(scaler.transform(new_data), columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                                                                   'BMI', 'DiabetesPedigreeFunction', 'Age'])

# Predict the outcome using the best estimator from GridSearchCV
y_pred_testing = best_estimator.predict(scaled_new_data)
print("New Diagnosis from New Data: ", y_pred_testing)

# **Dummy Classifier**

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_pred = dummy_clf.predict(X_test)

In [ ]:
print("Accuracy (Most Frequent Class Dummy Classifier):", accuracy_score(y_test, dummy_pred))